In [29]:
import numpy as np
import json
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm
import re
import pandas as pd
import string
import tensorflow as tf
import math

plt.style.use("fivethirtyeight")
%matplotlib inline

In [91]:
vocabulary_size = 254
embedding_size = 300
tweet_size = 140
batch_size = 100
num_sampled = 50
num_filters = 32
num_classes_for_prediction = 10

In [39]:
# Load the input data
twitter_data_dir = "../data/authorship_corpora/influencer_tweets.json"
json_data=open(twitter_data_dir).read()
twitter_data = json.loads(json_data)

# input_matrix = [np.zeros((vocab_size, tweet_size)) for i in np.arange(num_tweets)]

In [121]:
# Placeholders for inputs
train_inputs = tf.placeholder(tf.int32, shape=[batch_size, tweet_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, num_classes_for_prediction])

# one hot encode training labels

# input: vocab * n chars of specific tweets
embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embeddings.get_shape()

TensorShape([Dimension(254), Dimension(300)])

In [82]:
embed = tf.nn.embedding_lookup(embeddings, train_inputs)
embed = tf.reshape(embed, [100, 140, 300, 1])
embed.get_shape()

TensorShape([Dimension(100), Dimension(140), Dimension(300), Dimension(1)])

In [86]:
# Convolutional Layer #1
with tf.name_scope("conv-maxpool-1"):
    W = tf.Variable(tf.truncated_normal([2, embedding_size, 1, num_filters], stddev=0.05), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
    conv = tf.nn.conv2d(
        embed, 
        W, 
        strides=[1, 1, 1, 1], 
        padding="VALID", 
        name="conv1")
    h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
    pooled = tf.nn.max_pool(
        h,
        ksize=[1, 139, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool1")

In [88]:
pooled.get_shape()

TensorShape([Dimension(100), Dimension(1), Dimension(1), Dimension(32)])

In [112]:
# Fully connected layer 1
h_pool_flat = tf.reshape(pooled, [batch_size, num_filters])

# Multiply w * h_pool_flat and add bias terms
# 100*32 * 32*10 -> 100*10 + 10*1 = fc_1_output
with tf.name_scope("fc-1"):
    W = tf.Variable(tf.truncated_normal([num_filters, num_classes_for_prediction], stddev=0.05), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[num_classes_for_prediction]), name="b")
    fc_1_output = tf.nn.relu(tf.nn.xw_plus_b(h_pool_flat, W, b), name="fc-1-out")

fc_1_output.get_shape()

TensorShape([Dimension(100), Dimension(10)])

In [122]:
# cross entropy loss function
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
    _sentinel=None,
    labels=train_labels,
    logits=fc_1_output,
    dim=-1,
    name=None
))

In [123]:
# SGD optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

In [37]:
# Train data
    feed_dict = {train_inputs: inputs, train_labels: labels}
    _, cur_loss = session.run([optimizer, loss], feed_dict=feed_dict)

NameError: name 'generate_batch' is not defined

In [ ]:
# Visualize embeddings using TSNE
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.savefig(filename)

try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels, os.path.join(gettempdir(), 'tsne.png'))